In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
from dateutil import relativedelta

In [3]:
today = datetime.datetime.today()

In [48]:
hotspots = pd.read_csv('data/hotspots_test.csv')
users = pd.read_csv('data/users_test.csv')
connections = pd.read_csv('data/conns_test.csv')

### Get number of unique connections to hotspots

In [6]:
connections['connected_at'] = pd.to_datetime(connections['connected_at']).dt.tz_localize(None)

connections['connected_days_ago'] = (today - connections['connected_at']).dt.days
connections['connected_within_week'] = (connections['connected_days_ago'] <= 7).astype(int)
connections['connected_within_month'] = (connections['connected_days_ago'] <= 30).astype(int)
connections['connected_within_year'] = (connections['connected_days_ago'] <= 365).astype(int)

In [7]:
uniq_connections_within_week = connections[connections['connected_within_week']==1].groupby('hotspot_id').agg({
    'installation_id': lambda x: len(set(x))
}).rename(columns={'installation_id': 'uniq_connections_within_week'})

uniq_connections_within_month = connections[connections['connected_within_month']==1].groupby('hotspot_id').agg({
    'installation_id': lambda x: len(set(x))
}).rename(columns={'installation_id': 'uniq_connections_within_month'})

uniq_connections_within_year = connections[connections['connected_within_year']==1].groupby('hotspot_id').agg({
    'installation_id': lambda x: len(set(x))
}).rename(columns={'installation_id': 'uniq_connections_within_year'})

In [8]:
uniq_connections_all_time = connections.groupby('hotspot_id').agg({
    'installation_id': lambda x: len(set(x))
}).rename(columns={'installation_id': 'uniq_connections'})
          
          
uniq_connections = (uniq_connections_all_time
                    .join(uniq_connections_within_year)
                    .join(uniq_connections_within_month)
                    .join(uniq_connections_within_week)
                    .fillna(0).astype(int))

### Join to hotspots DF

In [137]:
conns_test1 = pd.DataFrame({
    'hotspot_id': [*range(len([0, 0, 0, 0, 0,
                 1, 1, 1,
                 2, 2, 2, 2, 
                 3]))],
    'uniq_connections': [0, 1, 5, 11, 20,
                         6, 8, 9,
                         1, 10, 14, 15,
                         60],
    'uniq_connections_within_year': [0, 1, 5, 10, 18,
                                     6, 7, 8,
                                     1, 10, 14, 15,
                                     55],
    'uniq_connections_within_month': [0, 1, 3, 7, 15,
                                      4, 6, 8,
                                      1, 9, 12, 14,
                                      30],
    'uniq_connections_within_week': [0, 0, 1, 3, 8,
                                     0, 1, 4,
                                     0, 4, 6, 8,
                                     10]
}).set_index('hotspot_id')

In [32]:
hotspots = hotspots.set_index('id').join(uniq_connections, how='left').reset_index()

for col in ['uniq_connections', 'uniq_connections_within_year', 'uniq_connections_within_month', 
            'uniq_connections_within_week']:
    hotspots[col].fillna(0, inplace=True)
    hotspots[col] = hotspots[col].astype(int)

In [33]:
hotspots['is_place'] = (hotspots['foursquare_id'].notna() | hotspots['google_place_id'].notna()).astype(int)
hotspots['created_at'] = pd.to_datetime(hotspots['created_at']).dt.tz_localize(None)

hotspots['created_days_ago'] = (today - hotspots['created_at']).dt.days
hotspots['created_within_week'] = (hotspots['created_days_ago'] <= 7).astype(int)
hotspots['created_within_month'] = (hotspots['created_days_ago'] <= 30).astype(int)

hotspots['is_good'] = (hotspots['score_v4'] > 0.6).astype(int)
hotspots['is_average'] = ((0.3 < hotspots['score_v4']) & (hotspots['score_v4'] <= 0.6)).astype(int)
hotspots['is_bad'] = (hotspots['score_v4'] <= 0.3).astype(int)

In [35]:
for col in ['uniq_connections', 'uniq_connections_within_year',
            'uniq_connections_within_month', 'uniq_connections_within_week']:
    for num in [1, 5, 10]:
        hotspots[f'{col}>{num}'] = (hotspots[col] > num).astype(int)

In [40]:
result = hotspots.groupby('owner_id').agg({'id': 'count', 
                                           'is_place': np.sum,
                                           'created_within_month': np.sum, 
                                           'created_within_week': np.sum, 
                                           'is_good': np.sum,
                                           'is_average': np.sum,
                                           'is_bad': np.sum,
                                           'uniq_connections>1': np.sum,
                                           'uniq_connections>5': np.sum,
                                           'uniq_connections>10': np.sum,
                                           'uniq_connections_within_year>1': np.sum,
                                           'uniq_connections_within_year>5': np.sum,
                                           'uniq_connections_within_year>10': np.sum,
                                           'uniq_connections_within_month>1': np.sum,
                                           'uniq_connections_within_month>5': np.sum,
                                           'uniq_connections_within_month>10': np.sum,
                                           'uniq_connections_within_week>1': np.sum,
                                           'uniq_connections_within_week>5': np.sum,
                                           'uniq_connections_within_week>10': np.sum
                                          }).rename(columns={'id': 'hotspots_created'})

In [41]:
result

,hotspots_created,is_place,created_within_month,created_within_week,is_good,is_average,is_bad,uniq_connections>1,uniq_connections>5,uniq_connections>10,uniq_connections_within_year>1,uniq_connections_within_year>5,uniq_connections_within_year>10,uniq_connections_within_month>1,uniq_connections_within_month>5,uniq_connections_within_month>10,uniq_connections_within_week>1,uniq_connections_within_week>5,uniq_connections_within_week>10
owner_id,,,,,,,,,,,,,,,,,,,
1118,42,37,0,0,32,1,6,12,3,2,12,3,2,2,0,0,0,0,0
1120,605,507,1,0,274,20,170,135,44,22,135,44,22,8,1,1,0,0,0
1359,63,59,0,0,33,4,14,19,3,2,19,3,2,0,0,0,0,0,0
3189,54,51,0,0,13,0,25,5,0,0,5,0,0,0,0,0,0,0,0
3425,8,7,0,0,1,0,6,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325248,31,19,0,0,11,0,18,4,0,0,4,0,0,0,0,0,0,0,0
329972,22,22,0,0,7,0,8,3,2,1,3,2,1,0,0,0,0,0,0
336356,40,34,0,0,11,2,20,6,1,0,6,1,0,0,0,0,0,0,0


In [60]:
def subtract_days_from_today(n_days=0):
    return datetime.datetime.today() - timedelta(days=n_days)

In [71]:
df_test1 = pd.DataFrame({
    'hotspot_id': [0, 0, 0, 0, 1, 1, 2, 2, 2], 
    'connected_at': [subtract_days_from_today(2), subtract_days_from_today(35), subtract_days_from_today(15), 
                     subtract_days_from_today(535),
                     subtract_days_from_today(1), subtract_days_from_today(55), 
                     subtract_days_from_today(35), subtract_days_from_today(535), subtract_days_from_today(535)
                    ],
    'installation_id': [0, 0, 0, 1, 0, 1, 0, 1, 2]
})

solution1 = pd.DataFrame({
    'hotspot_id': [0, 1, 2],
    'uniq_connections': [2, 2, 3],
    'uniq_connections_within_year': [1, 2, 1],
    'uniq_connections_within_month': [1, 1, 0],
    'uniq_connections_within_week': [1, 1, 0]
}).set_index('hotspot_id')

In [99]:
df_test2 = pd.DataFrame({
    'hotspot_id': [0, 0, 0, 0, 1, 1, 2, 2, 2], 
    'connected_at': [subtract_days_from_today(2), subtract_days_from_today(3), subtract_days_from_today(15), 
                     subtract_days_from_today(60),
                     subtract_days_from_today(400), subtract_days_from_today(410), 
                     subtract_days_from_today(1), subtract_days_from_today(1), subtract_days_from_today(20)
                    ],
    'installation_id': [0, 1, 2, 3, 
                        0, 1, 
                        0, 1, 1]
})

solution2 = pd.DataFrame({
    'hotspot_id': [0, 1, 2],
    'uniq_connections': [4, 2, 2],
    'uniq_connections_within_year': [4, 0, 2],
    'uniq_connections_within_month': [3, 0, 2],
    'uniq_connections_within_week': [2, 0, 2]
}).set_index('hotspot_id')

In [100]:
import json
import pickle

with open('/Users/vlad/PycharmProjects/wifimap_test_case/app/tests/cases/uniq_connections_test.pkl', 'wb') as f:
    pickle.dump([(df_test1, solution1), 
                 (df_test2, solution2)], f)

In [116]:
df_test1 = pd.DataFrame({
    'owner_id': [0, 0, 0, 0, 0,
                 1, 1, 1,
                 2, 2, 2, 2, 
                 3],
    'id': [*range(len([0, 0, 0, 0, 0,
                 1, 1, 1,
                 2, 2, 2, 2, 
                 3]))],
    'foursquare_id': ['some_place', 'some_place', np.nan, np.nan, np.nan, 
                      'some_place', np.nan, np.nan,
                      np.nan, np.nan, np.nan, np.nan,
                      'some_place'],
    'google_place_id': ['some_place', np.nan, 'some_place', np.nan, np.nan, 
                      'some_place', np.nan, 'some_place',
                      np.nan, 'some_place', np.nan, np.nan,
                      'some_place'],
    'created_at': [subtract_days_from_today(2), subtract_days_from_today(3), subtract_days_from_today(15), 
                   subtract_days_from_today(60), subtract_days_from_today(100),
                   subtract_days_from_today(10), subtract_days_from_today(400), subtract_days_from_today(410), 
                   subtract_days_from_today(1), subtract_days_from_today(1), subtract_days_from_today(20), 
                   subtract_days_from_today(400),
                   subtract_days_from_today(7)],
    'score_v4': [0.3, 0.5, 0.6, 0.8, 0.9,
                 0.1, np.nan, 0.8,
                 0.5, 0.7, 0.8, np.nan,
                 1]
})

In [ ]:
conns_test_init1 = pd.DataFrame({
    'hotspot_id': [1,
                   *[2 for _ in range(5)],
                   *[3 for _ in range(11)],
                   *[4 for _ in range(20)],
                   *[5 for _ in range(6)],
                   *[6 for _ in range(8)],
                   *[7 for _ in range(9)],
                   *[8 for _ in range(1)],
                   *[9 for _ in range(10)],
                   *[10 for _ in range(14)],
                   *[11 for _ in range(15)],
                   *[12 for _ in range(60)]
                  ], 
    'connected_at': [subtract_days_from_today(10),
                     subtract_days_from_today(1), 
                     *[subtract_days_from_today(10) for _ in range(2)], 
                     *[subtract_days_from_today(40) for _ in range(2)], 
                     
                     *[subtract_days_from_today(1) for _ in range(3)], 
                     *[subtract_days_from_today(10) for _ in range(4)],
                     *[subtract_days_from_today(40) for _ in range(3)],
                     subtract_days_from_today(400),
                     
                     *[subtract_days_from_today(1) for _ in range(8)],
                     *[subtract_days_from_today(10) for _ in range(7)],
                     *[subtract_days_from_today(40) for _ in range(3)],
                     *[subtract_days_from_today(400) for _ in range(2)],
                     
                     *[subtract_days_from_today(10) for _ in range(4)],
                     *[subtract_days_from_today(40) for _ in range(2)],
                     
                     *[subtract_days_from_today(1) for _ in range(1)],
                     *[subtract_days_from_today(10) for _ in range(5)],
                     *[subtract_days_from_today(40) for _ in range(1)],
                     *[subtract_days_from_today(400) for _ in range(1)],
                     
                     *[subtract_days_from_today(1) for _ in range(4)],
                     *[subtract_days_from_today(10) for _ in range(4)],
                     *[subtract_days_from_today(400) for _ in range(1)],
                     
                     *[subtract_days_from_today(10) for _ in range(1)],
                     
                     *[subtract_days_from_today(1) for _ in range(4)],
                     *[subtract_days_from_today(10) for _ in range(5)],
                     *[subtract_days_from_today(40) for _ in range(1)],

                     *[subtract_days_from_today(1) for _ in range(6)],
                     *[subtract_days_from_today(10) for _ in range(6)],
                     *[subtract_days_from_today(40) for _ in range(2)],
                     
                     *[subtract_days_from_today(1) for _ in range(8)],
                     *[subtract_days_from_today(10) for _ in range(6)],
                     *[subtract_days_from_today(40) for _ in range(1)],
                     
                     *[subtract_days_from_today(1) for _ in range(10)],
                     *[subtract_days_from_today(10) for _ in range(20)],
                     *[subtract_days_from_today(40) for _ in range(25)],
                     *[subtract_days_from_today(400) for _ in range(5)],
                    ],
    'installation_id': range(160)
})

In [159]:
final_solution1 = pd.DataFrame({
    'owner_id': [0, 1, 2, 3],
    'hotspots_created': [5, 3, 4, 1],
    'is_place': [3, 2, 2, 1],
    'created_within_month': [3, 1, 3, 1],
    'created_within_week': [2, 0, 2, 1],
    'is_good': [2, 1, 2, 1], 
    'is_average': [2, 0, 1, 0], 
    'is_bad': [1, 1, 0, 0],
    'uniq_connections>1': [3, 3, 3, 1], 
    'uniq_connections>5': [2, 3, 3, 1], 
    'uniq_connections>10': [2, 0, 2, 1],
    'uniq_connections_within_year>1': [3, 3, 3, 1], 
    'uniq_connections_within_year>5': [2, 3, 3, 1],
    'uniq_connections_within_year>10': [1, 0, 2, 1], 
    'uniq_connections_within_month>1': [3, 3, 3, 1],
    'uniq_connections_within_month>5': [2, 2, 3, 1], 
    'uniq_connections_within_month>10': [1, 0, 2, 1],
    
    'uniq_connections_within_week>1': [2, 1, 3, 1], 
    'uniq_connections_within_week>5': [1, 0, 2, 1],
    'uniq_connections_within_week>10': [0, 0, 0, 0]
}).set_index('owner_id')

In [163]:
with open('/Users/vlad/PycharmProjects/wifimap_test_case/app/tests/cases/grouping_test.pkl', 'wb') as f:
    pickle.dump([(df_test1, conns_test_init1, final_solution1)], f)